#   LangChain Tool 활용

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [ ]:
# Langsmith tracing 여부를 확인 (true: langsmith 추적 활성화, false: langsmith 추적 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

---

## **내장 도구 (Built-in Tool)**

- **내장 도구**는 시스템에 **기본적으로 포함**된 사전 정의된 도구들의 집합

- 파일 처리, 웹 검색, 코드 실행 등 **자주 사용되는 기본 기능**들을 즉시 활용 가능

- 별도의 구현 없이 **바로 사용**할 수 있어 개발 시간과 노력을 절약 가능

---

### **웹 검색(Tavily Search API)**

- **Tavily**는 AI 에이전트(LLM)를 위해 특별히 설계된 검색 엔진

- 검색 결과로 **제목, URL, 컨텐츠, 답변** 등 상세 정보 제공

- 개발자는 **월 1,000회** 무료 검색 쿼터 사용 가능

- **설치**: `pip install langchain-tavily` 또는 `uv add langchain-tavily`

- **인증키**: `TAVILY_API_KEY`

`(1) 도구 정의`

In [ ]:
from langchain_tavily import TavilySearch

search_web = TavilySearch(
    max_results=5,
    topic="general",
)

`(2) 도구 직접 호출`
- 자연어 쿼리를 도구에 전달

In [ ]:
result = search_web.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")
pprint(result)

`(3) ToolCall을 사용한 호출`
- 모델에서 생성된 `ToolCall`을 사용하여 도구를 호출

In [ ]:
from langchain_openai import ChatOpenAI

# 모델 생성
model = ChatOpenAI(model="gpt-4.1-mini")

# 모델에 도구 등록 
model_with_tools = model.bind_tools([search_web])

# 사용자 쿼리를 입력하여 ToolCall 생성
response = model_with_tools.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")

# ToolCall 출력 
pprint(response.tool_calls)

In [ ]:
# ToolCall을 사용하여 도구 실행
tool_msg = search_web.invoke(response.tool_calls[0])

# 도구 실행 결과 출력
pprint(tool_msg.content)

`(4) Agent와 연계`
- Tavily 도구를 언어 모델에 바인딩하고 사용자 입력을 처리하는 Agent 생성

In [ ]:
from langgraph.prebuilt import create_react_agent

# Agent 생성
agent = create_react_agent(model, [search_web])

# Agent에 사용자 입력 전달 및 실행
# stream_mode="values"로 설정하여 스트리밍 모드로 실행
user_input = "한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?"

for step in agent.stream(
    input={"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

---

##  **사용자 정의 도구 (Custom Tool)**


- **사용자 정의 도구**는 개발자가 직접 설계하고 구현하는 **맞춤형 함수나 도구**를 의미

- LLM이 호출할 수 있는 **고유한 기능**을 정의하여 특정 작업에 최적화된 도구 생성 가능

- 개발자는 도구의 **입력값, 출력값, 기능**을 자유롭게 정의하여 유연한 확장성 확보

---

### 1. **외부 API 연동** 

- LangChain에서는 **사용자 정의 도구**를 통해 외부 API와의 연동이 가능

- **Tool** 클래스를 상속받아 필요한 기능을 구현하며, `name`과 `description`을 필수로 정의

- 도구는 단일 기능을 수행하는 **함수 형태**로 구현되며, 입력과 출력이 명확하게 설정함

`(1) Yahoo Finance API` 

In [ ]:
#  yfinance 설치 : pip install yfinance 또는 uv add yfinance
import yfinance as yf

dat = yf.Ticker("MSFT")

In [ ]:
# 기업정보
dat.info

In [ ]:
# 2022년 1월 3일 ~ 4일의 데이터를 판다스 데이터프레임으로 출력 
result = dat.history(start="2022-01-03", end="2022-01-05") 
result

In [ ]:
# 인덱스 초기화 
result = result.reset_index()
result

In [ ]:
# 날짜 부분만 추출
result['Date'] = result['Date'].dt.strftime('%Y-%m-%d')

result

In [ ]:
# 데이터프레임을 딕셔너리로 변환
result_dict = result.to_dict(orient='records') 
result_dict

`(2) 데이터 연동 및 출력 포맷` 

In [ ]:
# 외부 API 연동하는 함수 (yfinance 사용)

from langchain_core.tools import ToolException
from typing import Dict, Optional
from datetime import datetime, timedelta
import yfinance as yf

def get_stock_price(symbol: str, date: Optional[str] = None) -> Dict:
    """yfiance 사용하여 특정 날짜의 주식의 가격 정보를 조회합니다."""

    if date and not is_valid_date(date):
        raise ToolException(f"잘못된 날짜 형식입니다: {date}")
    
    try:
        stock = yf.Ticker(symbol)
        # 특정 날짜의 주식 가격 정보 조회 
        if date:
            start = datetime.strptime(date, "%Y-%m-%d")
            end = start + timedelta(days=1)
            price = stock.history(start=start, end=end)

            # 가격 정보가 없으면 해날 날짜로부터 과거 5일간의 주식 가격 정보 조회
            if price.empty:
                end = start - timedelta(days=5)
                price = stock.history(start=end, end=start)

        # 특정 날짜가 없으면 최근 5일간의 주식 가격 정보 조회
        else:
            price = stock.history(period="5d")
            
        # 데이터프레임을 딕셔너리로 변환하여 반환 (가장 최근 날짜 데이터만 반환)
        df = price.reset_index()
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        return df.to_dict(orient='records')[-1]
    
    except Exception as e:
        raise ToolException(str(e))
    

def is_valid_date(date_str: str) -> bool:
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
        return True
    except ValueError:
        return False
    

# 함수 실행
result = get_stock_price("AAPL")

# 결과 출력
print(result)

In [ ]:
# 함수 실행 (날짜 지정)
result = get_stock_price("AAPL", "2025-01-03")
print(result)

`(3) StructuredTool 도구 변환` 

In [ ]:
from langchain_core.tools import StructuredTool

# StructuredTool로 도구 생성
stock_tool = StructuredTool.from_function(
    func=get_stock_price,
    name="stock_price_basic",
    description="yfinance를 사용하여 주식 가격 정보를 조회하는 도구입니다.",
)

# 도구 실행 (정상)
result = stock_tool.invoke({"symbol": "AAPL"})
print(result)

`(4) LLM 사용하여 도구 사용` 

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

# OpenAI GPT-4.1-mini 모델 사용
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 도구 실행 에이전트 생성
stock_agent = create_react_agent(llm, tools=[search_web, stock_tool])

# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 2025년 1월 3일 주식 가격을 알려줘")]}
)

for m in result['messages']:
    m.pretty_print()

In [ ]:
# 도구 실행 에인전트 생성 (return_direct=True)

stock_tool.return_direct = True
search_web.return_direct = True
stock_agent = create_react_agent(llm, tools=[search_web, stock_tool])

# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 2025년 1월 3일 주식 가격을 알려줘")]}
)

for m in result['messages']:
    m.pretty_print()

In [ ]:
# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 최신 뉴스를 알려줘")]}
)

for m in result['messages']:
    m.pretty_print()

---

### 2. **Naver 개발자 API** 를 도구로 사용

- 네이버 개발자 API(https://developers.naver.com/)에서 인증 권한 취득 (회원 가입 및 애플리케이션 등록 필요)
- 환경변수(.env)를 등록합니다. (**NAVER_CLIENT_ID**, **NAVER_CLIENT_SECRET**)
- 아래 정의된 네이버 뉴스 검색 도구(naver_news_search)를 사용하여 다음 과정을 수행합니다. 

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

`(1) 도구 정의`

In [ ]:
import requests, os
from langchain_core.tools import tool
from typing import Dict

@tool
def naver_news_search(
    query: str,
    ) -> Dict[Dict, int]:
    """네이버 검색 API를 사용하여 뉴스 검색 결과를 조회합니다."""

    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET")
    }
    params = {"query": query}

    response = requests.get(url, headers=headers, params=params)

    return {
        "data": response.json(),
        "status_code": int(response.status_code)
    }  #type: ignore

`(2) 도구 실행`

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

# 도구 목록
tools = [stock_tool, naver_news_search]  

# OpenAI GPT-4.1-mini 모델 사용
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 도구 실행 에이전트 생성
stock_agent = create_react_agent(llm, tools=tools)

# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 최근 주가는 어떻게 되나요? 최근 주가 분석에 대한 관련 기사를 찾아주세요.")]}
)

for m in result['messages']:
    m.pretty_print()

In [ ]:
pprint(result['messages'][-1].content)

`(3) 답변 생성`

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

# 도구 목록
tools = [stock_tool, naver_news_search]  

for tool in tools:
    tool.return_direct = False  # 도구 실행 결과를 직접 반환하지 않고, 에이전트가 처리하도록 설정

# OpenAI GPT-4.1-mini 모델 사용
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 도구 실행 에이전트 생성
stock_agent = create_react_agent(llm, tools=tools)

# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 최근 주가는 어떻게 되나요? 최근 주가 분석에 대한 관련 기사를 찾아주세요.")]}
)

for m in result['messages']:
    m.pretty_print()

In [ ]:
pprint(result['messages'][-1].content)

---
## **도구 호출 (Tool Calling)** 실습

- LangChain 문서를 확인하고, 내장 도구를 사용하여 도구 호출 과정을 이해합니다.

In [ ]:
# 여기에 코드를 작성하세요.